In [141]:
import pandas as pd
import string
import pickle

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
punctuation = []
for i in string.punctuation:
    punctuation.append(str(i))

In [155]:
df = pd.read_excel('../data/qa_data.xlsx')

In [156]:
df.head()

,Subject,Content,Source,Category
0,NaN,Wie werden die Geräte innerhalb meiner Wohnung...,FAQ (Q&A),Glasfaser
1,NaN,Gibt es spezifische Qualitätsmerkmale für Gesc...,FAQ (Q&A),Glasfaser
2,NaN,Welche Angebote können über das Glasfasernetz ...,FAQ (Q&A),Glasfaser
3,NaN,Wieso macht die (rasche) Vertragsunterschrift ...,FAQ (Q&A),Glasfaser
4,NaN,Wie lange dauert die Erschliessung in der Stad...,FAQ (Q&A),Glasfaser


In [157]:
df['Content'] = df['Content'].str.lower()
df['Content'] = df['Content'].str.replace(r'[^\w\s]+', '')

In [158]:
stop_words=set(stopwords.words("german"))

In [164]:
tokens = []
for i in df['Content']:
    i = i.replace('energie wasser bern', '')
    temp = []
    for word in word_tokenize(i):
        if word not in stop_words:
            if word not in blacklist:
                if word not in df.columns:
                    if not re.fullmatch('[0-9]*', word):
                        temp.append(word)
    tokens.append(temp)
df['Tokens'] = tokens

In [166]:
category_dict = {}
for i in set(df['Category']):
    temp = []
    for j in df.loc[(df['Category']== i)&(df['Source']=='FAQ (Q&A)')]['Tokens']:
        temp.extend(j)
    category_dict[i] = temp

In [167]:
cat_top10 ={}
cat_top10_inc_counts = {}
for i in category_dict.keys():
    fdist = FreqDist(category_dict[i])
    cat_top10_inc_counts[i] = fdist.most_common(10)
    cat_top10[i] = [i[0] for i in fdist.most_common(10)]
#pd.DataFrame(cat_top10).to_excel('top_10_words.xls')

In [12]:
cat_top20 ={}
cat_top20_inc_counts = {}
for i in category_dict.keys():
    fdist = FreqDist(category_dict[i])
    cat_top20_inc_counts[i] = fdist.most_common(20)
    cat_top20[i] = [i[0] for i in fdist.most_common(20)]
pd.DataFrame(cat_top20).to_excel('top_20_words.xls')
pd.DataFrame(cat_top20).to_csv('top_20_words.csv')

In [170]:
df_top10 = pd.DataFrame(cat_top10)

In [172]:
df_top10

,Glasfaser,Mobilitat,Umzug,Netz,Storungen,Internet,Wasser,Strom,Diverses,Kehricht,Warme
0,glasfasernetz,chf,umzug,kosten,klären,router,wasser,ökostrom,kunden,kehricht,erdgas
1,erschliessung,erdgas,melden,kunden,tel,ewbtv,schweiz,stromprodukt,smart,tonne,biogas
2,service,erdgasfahrzeuge,bringt,warum,ausgefallen,geräte,trinkwasser,kunden,zähler,pro,beim
3,eigentümer,gibt,meist,müssen,ab,ewbinternet,liter,produziert,meter,ab,etwa
4,anschluss,ebikes,bürokratischen,netzanschluss,handelt,kontrollieren,rund,kundinnen,teil,müssen,pro
5,angeschlossen,fahrzeuge,erledigungen,schweiz,nachbarn,nutzen,mineralwässer,bestellen,jahresrechnung,deponiert,jahr
6,angebote,akku,onlineformulare,trägt,störung,kundendienst,prozent,anteil,kunde,holz,preise
7,geschäftskunden,schweiz,erleichtern,swissgrid,rufen,fritzbox,hahnenwasser,hergestellt,kundinnen,erzeugt,beziehen
8,angeboten,km,abmeldung,schweizer,geplante,gerät,qualität,wählen,hydrospeicher,kehrichtmenge,kunden
9,finden,servicestelle,neue,netznutzung,beziehen,fall,natürlichen,stromverbrauch,energieversorger,zugenommen,ca


In [15]:
df_top10_scores = df_top10.copy()

In [173]:
all_scores = []
max_scores = []
all_categories = []
for i in df['Tokens']:
    score = dict.fromkeys(df_top10.columns, 0)
    for j, j_in in zip(df_top10.columns, list(df_top10.index)):
        for l in df_top10[j]:
            if l in i:
                score[j] += 10-j_in
    all_scores.append(score)
    max_scores.append(score[max(score, key=score.get)])
    all_categories.append(max(score, key=score.get))

In [174]:
df['max_scores'] = max_scores
df['all_scores'] = all_scores
df['all_categories'] = all_categories

In [181]:
df.head()

,Subject,Content,Source,Category,Tokens,max_scores,all_scores,all_categories,correct
0,NaN,wie werden die geräte innerhalb meiner wohnung...,FAQ (Q&A),Glasfaser,"[geräte, innerhalb, angeschlossen, endkundenan...",20,"{'Glasfaser': 20, 'Mobilitat': 0, 'Umzug': 0, ...",Glasfaser,False
1,NaN,gibt es spezifische qualitätsmerkmale für gesc...,FAQ (Q&A),Glasfaser,"[gibt, spezifische, qualitätsmerkmale, geschäf...",10,"{'Glasfaser': 10, 'Mobilitat': 9, 'Umzug': 0, ...",Glasfaser,False
2,NaN,welche angebote können über das glasfasernetz ...,FAQ (Q&A),Glasfaser,"[angebote, glasfasernetz, genutzt, sowohl, pri...",50,"{'Glasfaser': 50, 'Mobilitat': 0, 'Umzug': 0, ...",Glasfaser,False
3,NaN,wieso macht die rasche vertragsunterschrift na...,FAQ (Q&A),Glasfaser,"[wieso, macht, rasche, vertragsunterschrift, e...",10,"{'Glasfaser': 10, 'Mobilitat': 0, 'Umzug': 0, ...",Glasfaser,False
4,NaN,wie lange dauert die erschliessung in der stad...,FAQ (Q&A),Glasfaser,"[lange, dauert, erschliessung, stadt, gesamte,...",30,"{'Glasfaser': 30, 'Mobilitat': 0, 'Umzug': 0, ...",Glasfaser,False


In [175]:
correct = [x == y for x,y in zip(df['Category'], df['max_scores'])]

In [176]:
df['correct'] = correct

In [177]:
sum(df.loc[df['Source']=='FAQ (Q&A)']['correct'])/len(df.loc[df['Source']=='FAQ (Q&A)'])

0.0

In [128]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

path = 'google_credentials.json' 
client = language.LanguageServiceClient.from_service_account_json(path)

nltk.download('stopwords')
df_top10 = pd.read_excel('top_10_words.xls')

def preprocess(text):
    text = text.lower()
    text = text.replace(r'[^\w\s]+', '')
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

def get_score(tokens):
    score = dict.fromkeys(df_top10.columns, 0)
    for j, j_in in zip(df_top10.columns, list(df_top10.index)):
        for l in df_top10[j]:
            if l in tokens:
                score[j] += 10-j_in
    category = (max(score, key=score.get))
    score = score[max(score, key=score.get)]
    return score, category

def google_sentiment(text):
    # Score is positive or negative sentiment
    # Magnitude is how much the text displays that
    document = language.types.Document(
            content=text,
            type=language.enums.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(document=document)
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    if (score <= 0.0):
        ex_negative = True
    else:
        ex_negative = False
    return ex_negative

def packaged_results(id, timestamp, message, user_name, contact_details):  #user_id, etc just given as examples
    category_score, category = get_score(preprocess(message))
    extreme_negative = google_sentiment(message)
    return {'id':id, 'timestamp':timestamp,'user_name':user_name, 'message': message, 'category': category, 'category_score': score, 'extreme_negative': extreme_negative}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [130]:
packaged_results('id','time',df_email['Mail'].loc[2], 'user_name', 'contact_details')

{'id': 'id',
 'timestamp': 'time',
 'user_name': 'user_name',
 'message': 'Guten Abend \n\nNachdem ich seit dem ersten Tag diverse Probleme mit ewbTV habe und mich tagtäglich ärgere, dass etwas nicht funktioniert, und nachdem ich schon mehrmals mit dem technischen Support nach einer Lösung gesucht habe, folgte nun der bisherige Höhepunkt: Ich schaltete den TV und und stand vor einem Schirm, der nach Kundennummer und Aktivierungscode fragt. \nIch wüsste nicht, wo ich so einen Aktivierungscode her haben sollte. Die untenstehenden Angaben habe ich versucht, das hat nicht geklappt. Am telefonischen Kundendienst sagt man mir, man könne mir nicht helfen, ich solle am Montag wieder anrufen. Das ist natürlich kein Support, wie man ihn heutzutage erwartet. \n\nDer techn. Support hat mir letzten Dienstag versprochen, eine neue TV-Box zu schicken, vielleicht klappt es mit dieser ja besser. Aber die ist noch nicht eingetroffen ... evtl. hat man mir die jetzige Box schon deaktiviert? \n\nIrgendwie 

In [37]:
def preprocess(text):
    text = text.lower()
    text = text.replace(r'[^\w\s]+', '')
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

In [47]:
def get_score(tokens):
    score = dict.fromkeys(df_top10.columns, 0)
    for j, j_in in zip(df_top10.columns, list(df_top10.index)):
        for l in df_top10[j]:
            if l in tokens:
                score[j] += 10-j_in
    category = (max(score, key=score.get))
    score = score[max(score, key=score.get)]
    return score, category

In [44]:
def get_multiple(series):
    categories = []
    scores = []
    for i in series:
        score, category =get_score(preprocess(i))
        scores.append(score)
        categories.append(category)
    return categories, scores

In [48]:
score, category =get_score(preprocess(df_email['Mail'].loc[0]))

In [49]:
score

10

In [50]:
category

'Storungen'

# Testing

In [132]:
df_email = pd.read_excel('../data/Mails.xlsx')

In [139]:
df_email.head(50)

,Betreff,Mail,Zuständig,Antwort,Kategorie,Category,categories,correct
0,Wechsel Stromprodukt,Sehr geehrte Damen und Herren\n\nGerne möchte ...,Allgemein,Verweis Kundenportal,Strom,Strom,Storungen,False
1,Re: Aufschaltbestätigung,"Guten Tag\n\nAlles bestens soweit, herzlichen ...",Internet,FAQ,"Internet, TV",Internet,Unnamed: 0,False
2,Re: Aufschaltbestätigung,Guten Abend \n\nNachdem ich seit dem ersten Ta...,Internet,NaN,"Internet, TV, Störungen",Internet,Mobilitat,False
3,NaN,Sehr geehrte Damen und Herren\n \nLeider sehe ...,Allgemein,NaN,"Strom, Wärme",Strom,Mobilitat,False
4,DRINGEND: Ihre Anfrage betreffend Glasfaserint...,Sehr geehrte Damen und Herren\n\nIch hab’s wir...,Beschwerdemanagement,NaN,"Internet, Glasfaser",Internet,Glasfaser,False
5,NaN,Guten Tag \n \nBesten Dank für schnelle Auscha...,Internet,FAQ,"Internet, TV",Internet,Netz,False
6,Re: Aufschaltbestätigung,"Sehr Geehrte Damen und Herren,\nbedeutet es, d...",Internet,FAQ,"Internet, TV",Internet,Mobilitat,False
7,"Anfrage Musterstrasse 2, 3000 Bern (Faktor: Ve...",Sehr geehrte Damen und Herren\n\nBei der Durch...,Allgemein,FAQ,Wärme,Warme,Mobilitat,False
8,Ordentliche Kündigung meines Internet Abos,"Guten Tag,\n\nIch möchte mein Internetzugang s...",Internet,FAQ,Internet,Internet,Umzug,False
9,Fragen zum Wechsel zum Einheitstarif,Sehr geehrte Damen und Herren\n\nWir haben die...,Allgemein,FAQ,Strom,Strom,Storungen,False


In [134]:
categories, scores = get_multiple(df_email['Mail'])

In [135]:
df_email['categories'] = categories

In [35]:
df_email.to_excel('email_cats.xls')

In [136]:
df_email['correct'] = [x == y for x,y in zip(df_email['Category'], df['Category'])]

In [137]:
df_email['correct'].value_counts()

False    79
True      3
Name: correct, dtype: int64

# Sentiment Analysis

In [48]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [49]:
#Setup

path = '/Users/chris/OneDrive/flatiron/own_work/attest/google_credentials.json' 
client = language.LanguageServiceClient.from_service_account_json(path)

In [51]:
google_sentiment(df_email['Mail'].loc[0])

(0.6000000238418579, 2.5)

In [110]:
def google_sentiment(text):
    # Score is positive or negative sentiment
    # Magnitude is how much the text displays that
    document = language.types.Document(
            content=text,
            type=language.enums.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(document=document)
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    return score, magnitude

In [107]:
neg_emails = [2, 4, 14, 41]
neu_emails = [0, 5, 7, 9]

In [111]:
google_sentiment(df_email['Mail'].loc[2])

(-0.10000000149011612, 5.300000190734863)

In [117]:
neg_sentiments = {}
neg_scores = []
neg_mag = []
for i in neg_emails:
    score, magnitude = google_sentiment(df_email['Mail'].loc[i])
    neg_sentiments[i] = {'text':df_email['Mail'].loc[i], 'score':score, 'magnitude':magnitude}
    neg_scores.append(score)
    neg_mag.append(magnitude)

In [118]:
neu_sentiments = {}
neu_scores = []
neu_mag = []
for i in neu_emails:
    score, magnitude = google_sentiment(df_email['Mail'].loc[i])
    neu_sentiments[i] = {'text':df_email['Mail'].loc[i], 'score':score, 'magnitude':magnitude}
    neu_scores.append(score)
    neu_mag.append(magnitude)

In [125]:
neg_sentiments    

{2: {'text': 'Guten Abend \n\nNachdem ich seit dem ersten Tag diverse Probleme mit ewbTV habe und mich tagtäglich ärgere, dass etwas nicht funktioniert, und nachdem ich schon mehrmals mit dem technischen Support nach einer Lösung gesucht habe, folgte nun der bisherige Höhepunkt: Ich schaltete den TV und und stand vor einem Schirm, der nach Kundennummer und Aktivierungscode fragt. \nIch wüsste nicht, wo ich so einen Aktivierungscode her haben sollte. Die untenstehenden Angaben habe ich versucht, das hat nicht geklappt. Am telefonischen Kundendienst sagt man mir, man könne mir nicht helfen, ich solle am Montag wieder anrufen. Das ist natürlich kein Support, wie man ihn heutzutage erwartet. \n\nDer techn. Support hat mir letzten Dienstag versprochen, eine neue TV-Box zu schicken, vielleicht klappt es mit dieser ja besser. Aber die ist noch nicht eingetroffen ... evtl. hat man mir die jetzige Box schon deaktiviert? \n\nIrgendwie läuft einfach alles schief, seit dem ersten Tag. Nein, schon 

In [142]:
blacklist = pickle.load(open('../Categorizing/blacklist.p', 'rb'))

In [144]:
len(blacklist)

100

In [150]:
import pandas as pd
import pickle
import re

TOP_WORDS = './../NLP/top_20_words.csv'

# TODO: Remove category names from important words of other category


def do():
    words = pd.read_csv(TOP_WORDS)
    blacklist = pickle.load(open('../Categorizing/blacklist.p', 'rb'))
    # print(words.head(5), '\n' + '-' * 50 + '\n', words.describe())

    categories = words.columns
    parsed = {}
    for cat in categories:
        parsed[cat] = []
        for word in words[cat]:
            if not isinstance(word, str):
                continue
            # Remove common words
            if word in blacklist:
                continue
            # Don't take other categories names as key word
            if word in categories and not word == cat:
                continue
            # And we also don't want to accept integers as 'words'
            if re.fullmatch('[0-9]*', word):
                continue
            parsed[cat].append(word)

    maxlen = min([len(l) for l in parsed])
    take_top = min(maxlen, 15)
    parsed = [l[:take_top] for l in parsed]

    return parsed

In [151]:
df = do()

In [152]:
df

['Unna',
 'Glas',
 'Mobi',
 'Umzu',
 'Netz',
 'Stor',
 'Inte',
 'Wass',
 'Stro',
 'Dive',
 'Kehr',
 'Warm']